In [4]:
%run ttn_handler.ipynb # The run behaves like an import. This is needed since this is not a .py file
%run map_handler.ipynb
%run gui.ipynb
%run storage.ipynb

# TTN access info
app_id = 'befinitiv-testnode'
access_key = 'ttn-account-****'


# gets called from TTN
def uplink_callback(msg, client):
    msgdict = ttn_handler.msg_to_dict(msg)
    if g.w_live.value:
        s.append_msg(msgdict)
        update_map(msgdict)
        g.update(s.msg_count(), s.msg_count()-1, ttn_handler.msg_timestamp(msgdict))

# gets called from slider
def replay_callback(msg_idx):
    msg = s.get_msg(msg_idx)
    update_map(msg)
    g.update(s.msg_count(), msg_idx, ttn_handler.msg_timestamp(msg))
    
# gets called from "Load TTN file"
def load_ttn_callback(inbytes):
    s.load(inbytes)
    replay_callback(0)
    
# clears map and adds red circles for each gateway that received our msg
def update_map(msg):
    map_hnd.clear_circles()
    for gw in msg['metadata']['gateways']:
        loc = ttn_hnd.gtw_location(gw['gtw_id'])
        if loc != None:
            map_hnd.add_circle((*loc, ttn_hnd.rssi_to_dist(gw['rssi'])))

    map_hnd.center_map(loc)
      
map_hnd = map_handler(zoom=2)
ttn_hnd = ttn_handler(app_id, access_key, uplink_callback)
s = storage()
g = gui(replay_callback, s.save_file, load_ttn_callback, map_hnd.load_gpx_track)


Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…